# 1.Install necessary packages
- segment-geospatial
- leafmap
- localtileserver

In [3]:
%pip install segment-geospatial leafmap localtileserver

  Using cached segment_geospatial-0.12.4-py2.py3-none-any.whl.metadata (11 kB)
  Using cached leafmap-0.42.13-py2.py3-none-any.whl.metadata (16 kB)
  Using cached localtileserver-0.10.6-py3-none-any.whl.metadata (5.2 kB)
  Using cached fiona-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (56 kB)
  Using cached ipympl-0.9.7-py3-none-any.whl.metadata (8.7 kB)
  Using cached patool-4.0.0-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached rasterio-1.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
  Using cached rioxarray-0.18.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached sam2-1.1.0.tar.gz (152 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta

In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

# 2.Create interactive map

In [2]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19) # The longitude & latitude coordinates of the map's center
m.add_basemap("SATELLITE")  # Set the basemap to "SATELLITE", you can change the basemap type using the top right "ToolBar" button
m  # display the map

Map(center=[29.67684, -95.369222], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

**Setting the Region of Interest (ROI) for the segmentation**

In [6]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3714, 29.6746, -95.3673, 29.6779] # The default ROI

In [5]:
m.user_roi_bounds()  # Display the ROI

# 3.Download the ROI map in .tif format


In [7]:
image = "ROImage.tif"  # the name of the map

In [ ]:
# image = 'path/image.tif'  # You can also upload your own image data for segmentation

Using ``tms_to_geotiff()`` function to convert thr image to the .tif format

In [8]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 001/156
Downloaded image 002/156
Downloaded image 003/156
Downloaded image 004/156
Downloaded image 005/156
Downloaded image 006/156
Downloaded image 007/156
Downloaded image 008/156
Downloaded image 009/156
Downloaded image 010/156
Downloaded image 011/156
Downloaded image 012/156
Downloaded image 013/156
Downloaded image 014/156
Downloaded image 015/156
Downloaded image 016/156
Downloaded image 017/156
Downloaded image 018/156
Downloaded image 019/156
Downloaded image 020/156
Downloaded image 021/156
Downloaded image 022/156
Downloaded image 023/156
Downloaded image 024/156
Downloaded image 025/156
Downloaded image 026/156
Downloaded image 027/156
Downloaded image 028/156
Downloaded image 029/156
Downloaded image 030/156
Downloaded image 031/156
Downloaded image 032/156
Downloaded image 033/156
Downloaded image 034/156
Downloaded image 035/156
Downloaded image 036/156
Downloaded image 037/156
Downloaded image 038/156
Downloaded image 039/156
Downloaded image 040/156


Display the downloaded image

In [9]:
m.layers[-1].visible = False  # 关闭卫星底图,从而更直观地查看影像位置
m.add_raster(image, layer_name="Image")
m

Map(bottom=27757313.0, center=[29.676250000000003, -95.36935], controls=(ZoomControl(options=['position', 'zoo…

# 4.Segment the image using SAM

① Initiate the parameters of the model

In [10]:
sam = SamGeo(
    model_type="vit_h",  # The default model type of SAM
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:11<00:00, 224MB/s]


② Image segmentation

- It is **highly recommended** to run under GPU to speed up the efficiency, the running time will be different according to the size of the image.
- The ``batch`` parameter is set to **TRUE**, which means to segment the image in "batches" in order to prevent reduction in operating efficiency
- Let the ``erosion_kernel`` cell be 3×3, and the value range of the greyscale map (``mask_multiplier``) be 0-255

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

③ Save the segment result to a shapefile

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

④ Visualizing the segmentation

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(shapefile, layer_name="Vector", style=style)
m